<a href="https://colab.research.google.com/github/cacbondioxit/2022-Spring-ML-proj/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pathlib, os
from functools import partial
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from scipy.stats import norm, skew
from scipy.special import boxcox1p

# '/device:GPU:0'
print(tf.__version__)

import matplotlib.font_manager as fm
from matplotlib import rc
# a = r"C:\Users\ShinJunhyung\AppData\Local\Microsoft\Windows\Fonts\NanumGothic.ttf"
# font_path = r"C:\Windows\Fonts\NanumGothicLight.ttf"
# FONT = fm.FontProperties(fname=a).get_name()
# rc('font', family=FONT)

2.8.2


In [ ]:
f2015 = "/content/drive/MyDrive/기쁨코인/dataset/최종 table/TABLE_2015.csv"
f2016 = "/content/drive/MyDrive/기쁨코인/dataset/최종 table/TABLE_2016.csv"
f2017 = "/content/drive/MyDrive/기쁨코인/dataset/최종 table/TABLE_2017.csv"

In [ ]:
data1 = pd.read_csv(f2015, index_col=0)
data2 = pd.read_csv(f2016, index_col=0)
data3 = pd.read_csv(f2017, index_col=0)
data=pd.concat([data1, data2], axis=0)
data=pd.concat([data, data3], axis=0)

In [ ]:
data.head()

,building t,case num,address,Latitude,Longitude,size,appraiser,lowest,success,bid_date,...,공공행정국방및사회보장행정,공공행정국방및사회보장행정 증감률,교육서비스업,교육서비스업 증감률,보건업 및 사회복지 서비스업,보건업 및 사회복지 서비스업 증감률,문화 및 기타서비스업,문화 및 기타서비스업 증감률,순생산물세,순생산물세 증감률
0,오피스텔,2015 타경 12810,서울 강남구 대치동 889-47 비동 3층 317호,37.503835,127.050223,건물 38.71㎡(11.71평) / 대지권 7.75㎡(2.34평),2.020000e+08,161600000,1.911000e+08,2015/12/29,...,275093,-16.1,1588177,0.0,2419185,6.4,1586264,-3.7,5429338,4.8
1,종교시설,2015 타경 2677,서울 강남구 자곡동 288,37.479831,127.102842,건물 797.09㎡(241.12평) / 토지 655㎡(198.14평) / 제시외 1...,3.872439e+09,3097951000,3.133300e+09,2015/12/24,...,275093,-16.1,1588177,0.0,2419185,6.4,1586264,-3.7,5429338,4.8
2,근린상가,2015 타경 4680,서울 강남구 논현동 108 논현웰스톤 1층 162호,37.518417,127.038003,건물 129.21㎡(39.09평) / 대지권 40.77㎡(12.33평),7.840000e+08,627200000,7.017000e+08,2015/12/24,...,275093,-16.1,1588177,0.0,2419185,6.4,1586264,-3.7,5429338,4.8
3,근린상가,2015 타경 1056,서울 강남구 역삼동 815 점프밀라노 9층 901호,37.501275,127.026641,건물 86.45㎡(26.15평) / 대지권 17.95㎡(5.43평),4.300000e+08,430000000,9.950000e+08,2015/11/26,...,275093,-16.1,1588177,0.0,2419185,6.4,1586264,-3.7,5429338,4.8
4,근린시설,2015 타경 2585,서울 강남구 청담동 7,37.523254,127.047221,건물 1369.19㎡(414.18평) / 토지 431.9㎡(130.65평) / 제시...,8.358750e+09,8358750000,9.311000e+09,2015/10/14,...,275093,-16.1,1588177,0.0,2419185,6.4,1586264,-3.7,5429338,4.8


In [ ]:
data.columns

Index(['building t', 'case num', 'address', 'Latitude', 'Longitude', 'size',
       'appraiser', 'lowest', 'success', 'bid_date',
       ...
       '공공행정국방및사회보장행정', '공공행정국방및사회보장행정 증감률', '교육서비스업', '교육서비스업 증감률',
       '보건업 및 사회복지 서비스업', '보건업 및 사회복지 서비스업 증감률', '문화 및 기타서비스업 ',
       '문화 및 기타서비스업 증감률', '순생산물세', '순생산물세 증감률'],
      dtype='object', length=136)

In [ ]:
data.loc[:, 'building t'].unique()

array(['오피스텔', '종교시설', '근린상가', '근린시설', '문화/집회시설', '숙박시설', '사무실', '상업용기타',
       '아파트형공장', '의료시설', '공장', '창고', '자동차시설', '주유소', '노유자 시설'],
      dtype=object)

In [ ]:
#size 피처 평수만 가져오기
data['size']=data['size'].str.split(' ').str[1]
data['size']=data['size'].str.split('㎡').str[0]
data= data.astype({'size':'float'})
data=data.dropna()

#1km변수만 가져옴
data_1=data.filter(regex='_1')

#앞의 변수중 필요한 변수 select
list_fo= [0, 5, 8]
data_2= data.iloc[:, list_fo]

#뒤의 변수중 다중공선성 작은 변수 select
list_gu=[ 86, 87, 90, 93, 97, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134]
data_3=data.iloc[:, list_gu]
all=pd.concat([data_2, data_1], axis=1)
all=pd.concat([all, data_3], axis=1)

In [ ]:

all

,building t,size,success,일반게임제공업_1km,골프연습장업_1km,공연장_1km,단란주점_1km,노래연습장_1km,당구장_1km,숙박업_1km,...,숙박 및 음식점업,정보통신업,금융 및 보험업,부동산업,사업서비스업,공공행정국방및사회보장행정,교육서비스업,보건업 및 사회복지 서비스업,문화 및 기타서비스업,순생산물세
0,오피스텔,38.71,1.911000e+08,5,65,2,46,74,82,33,...,1591091,6968728,2583780,8593940,12316944,275093,1588177,2419185,1586264,5429338
1,종교시설,797.09,3.133300e+09,1,6,0,1,4,6,0,...,1591091,6968728,2583780,8593940,12316944,275093,1588177,2419185,1586264,5429338
2,근린상가,129.21,7.017000e+08,1,42,6,82,27,60,11,...,1591091,6968728,2583780,8593940,12316944,275093,1588177,2419185,1586264,5429338
3,근린상가,86.45,9.950000e+08,9,62,9,83,120,125,55,...,1591091,6968728,2583780,8593940,12316944,275093,1588177,2419185,1586264,5429338
4,근린시설,1369.19,9.311000e+09,1,27,1,20,14,19,7,...,1591091,6968728,2583780,8593940,12316944,275093,1588177,2419185,1586264,5429338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,오피스텔,56.59,1.670000e+08,21,11,12,31,135,54,110,...,226931,423928,205699,1159865,784987,263173,818461,526883,192765,431302
762,근린상가,27.95,1.510000e+07,21,11,10,30,138,57,109,...,226931,423928,205699,1159865,784987,263173,818461,526883,192765,431302
763,근린상가,27.95,1.610000e+07,21,11,10,30,138,57,109,...,226931,423928,205699,1159865,784987,263173,818461,526883,192765,431302
764,근린상가,9.25,1.664000e+07,2,13,12,15,38,29,19,...,226931,423928,205699,1159865,784987,263173,818461,526883,192765,431302


In [ ]:
all.isna().sum().sum()

0

In [ ]:
y_data= all['success']
all=all.drop('success', axis=1)

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(all, y_data, test_size=0.2)
ntrain=x_train.shape[0]
ntest=x_test.shape[0]

In [ ]:
#target 데이터 로그 정규화
y_train=np.log1p(y_train)

#feature중 왜도가 높은 피처 로그정규화
all_x=pd.concat((x_train, x_test)).reset_index(drop=True)

numeric_feats= all_x.dtypes[all_x.dtypes!='object'].index

skewed_feats= all_x[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness= pd.DataFrame({'Skew': skewed_feats})
skewness.head(10)

skewness= skewness[abs(skewness)>0.75]

skewed_features= skewness.index
lam=0.15
for feat in skewed_features:
    all_x[feat]=boxcox1p(all_x[feat], lam)

#범주형 변수 변환
all_x=pd.get_dummies(all_x)

train=all_x[:ntrain]
test=all_x[ntrain:]

n_folds = 5

#CV
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=10).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring='neg_mean_squared_error', cv = kf))
    return(rmse)

In [ ]:
y_test=np.log1p(y_test) # y_test도 스케일링

In [ ]:
train.isna().sum()

size                       0
일반게임제공업_1km                0
골프연습장업_1km                 0
공연장_1km                    0
단란주점_1km                   0
노래연습장_1km                  0
당구장_1km                    0
숙박업_1km                    0
유흥주점영업_1km                 0
음식점_1km                    0
인터넷컴퓨터게임시설제공업_1km          0
학원_1km                     0
학교_1km                     0
카페_1km                     0
체력단련장_1km                  0
의원_1km                     0
병원_1km                     0
대규모점포_1km                  0
관광극장유흥업_1km                0
인구밀도                       0
세대                         0
한국인 합계                     0
등록외국인 합계                   0
65세이상고령자                   0
지역내총생산(당해년가격)              0
지역내총생산(2015년 기준년 연쇄가격)     0
지역내총부가가치(2015년 기준년연쇄가격)    0
농업 임업 및 어업                 0
제조업                        0
전기가스증기 및  공기조절 공급업         0
건설업                        0
도매 및 소매업                   0
운수 및 창고업                   0
숙박 및 음식점업                  0
정보통신업         

In [ ]:
len(train.columns)

58

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(len(train.columns), activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.Adam(0.0001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse', 'msle'])
  return model

model = build_model()

In [ ]:
example_batch = train[:10]
example_result = model.predict(example_batch)
example_result
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 58)                3422      
                                                                 
 dense_1 (Dense)             (None, 32)                1888      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 5,855
Trainable params: 5,855
Non-trainable params: 0
_________________________________________________________________


In [ ]:
tf.random.set_seed(42)

EPOCHS = 500
# EPOCHS = 2000

history = model.fit(
    train, y_train,
    epochs=EPOCHS, validation_split = 0.2, verbose=0)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail(10)

,loss,mae,mse,msle,val_loss,val_mae,val_mse,val_msle,epoch
490,0.401228,0.478447,0.401228,0.001057,0.450522,0.478817,0.450522,0.001154,490
491,0.382871,0.462207,0.382871,0.001009,0.449033,0.481192,0.449033,0.001148,491
492,0.381444,0.456805,0.381444,0.001007,0.455802,0.484184,0.455802,0.001171,492
493,0.388411,0.462155,0.388411,0.001027,0.456986,0.488855,0.456986,0.001172,493
494,0.384500,0.460136,0.384500,0.001013,0.469970,0.492787,0.469970,0.001204,494
495,0.384531,0.460917,0.384531,0.001015,0.486507,0.505847,0.486507,0.001257,495
496,0.383886,0.460304,0.383886,0.001015,0.460170,0.489402,0.460170,0.001172,496
497,0.389548,0.462166,0.389548,0.001030,0.455798,0.484004,0.455798,0.001164,497
498,0.378109,0.458300,0.378109,0.000999,0.449952,0.480811,0.449952,0.001155,498
499,0.385176,0.462157,0.385176,0.001018,0.477035,0.508810,0.477035,0.001212,499


In [ ]:
hist.iloc[499, 4]

0.5066669583320618

In [ ]:
model.predict(test)

array([[18.580536],
       [21.421494],
       [19.516369],
       [18.912222],
       [19.969803],
       [19.91949 ],
       [19.240776],
       [18.359774],
       [17.453476],
       [19.246708],
       [17.862642],
       [17.614592],
       [18.785847],
       [20.9908  ],
       [25.657055],
       [17.240902],
       [18.566637],
       [20.287188],
       [25.339588],
       [20.92562 ],
       [19.752066],
       [21.77351 ],
       [17.67468 ],
       [20.347124],
       [17.403975],
       [22.46022 ],
       [17.896473],
       [17.937302],
       [18.624472],
       [19.889214],
       [18.522879],
       [19.164223],
       [19.603096],
       [19.847597],
       [19.47882 ],
       [23.6678  ],
       [16.985218],
       [21.024506],
       [18.063702],
       [18.635319],
       [18.340057],
       [18.063702],
       [20.480082],
       [18.063702],
       [20.229557],
       [18.319828],
       [20.086124],
       [17.630005],
       [18.523365],
       [17.751213],


In [ ]:
y_test

95      17.624393
994     21.345030
315     19.382404
964     18.714096
293     20.030319
          ...    
1105    16.301250
231     19.275096
834     19.395911
729     19.499090
158     18.469471
Name: success, Length: 560, dtype: float64

In [ ]:
RMSLE = np.sqrt(mean_squared_log_error(y_test, model.predict(test)))

RMSLE

0.036614703901903176

In [ ]:
print('Neural Network cv score: {:.4f} '.format(hist.iloc[499, 4]))
print('Neural Network RMSLE: {:.4f} '.format(RMSLE))

Neural Network cv score: 0.4770 
Neural Network RMSLE: 0.0366 
